In [1]:
import gymnasium as gym
from gymnasium.spaces import Discrete, Box, Sequence, Dict
import numpy as np
import pandas as pd
from pandarallel import pandarallel
from src.commons import methods
import statistics

from src.rl_environments.env_impr3 import EnvImpr3
from src.model_approaches.analytical.oracle_fix_n import OracleFix_n
from src.model_approaches.analytical.oracle_var_n import OracleVar_n
from src.model_approaches.analytical.pto_fix_n import PTO_Fix_n
from src.model_approaches.analytical.pto_var_n import PTO_Var_n
from src.model_approaches.rl.ppo import PPO

from src.commons import generate_data, methods
from src.commons.constants import AlphaEstimator, TravelTimeDist
import os
import pickle
from pathlib import Path

from collections import OrderedDict

## Train Config

In [2]:
PARAM_ESTIMATOR = AlphaEstimator.MAX_LIKELI

TRAIN_CONFIG = {
    'alpha_range': np.arange(0.1, 7.0, 0.1).tolist(),
    'beta_range': np.arange(0.1, 5.0, 0.1).tolist(),
    'h_range': [0.5],
    'c_range': [25],
    'total': list(range(5, 40)),
    'travel_time': TravelTimeDist.UNIFORM,
    'param_estimator': PARAM_ESTIMATOR,
}

PPO_TIME_STEPS = 2_000_000
PPO_ENV_CLASS = EnvImpr3


## Model Options

In [3]:
oracle_models = [OracleVar_n(), OracleFix_n(n=3), OracleFix_n(n=5)]
user_models = [PPO(EnvClass=EnvImpr3, config=TRAIN_CONFIG, timesteps=PPO_TIME_STEPS),
               PTO_Var_n(param_estimator=PARAM_ESTIMATOR),
               PTO_Fix_n(n=3, param_estimator=PARAM_ESTIMATOR),
               PTO_Fix_n(n=5, param_estimator=PARAM_ESTIMATOR)]

cost_results_df = pd.DataFrame(columns=['Test Name', 'PPO', 'PTO_Var_n', 'Oracle_Var_n', 'PTO_Fix_n_3', 'Oracle_Fix_n_3', 'PTO_Fix_n_5', 'Oracle_Fix_n_5'])
observed_n_results_df = pd.DataFrame(columns=['Test Name', 'PPO', 'PTO_Var_n', 'Oracle_Var_n'])

[INFO] Loaded trained PPO model from:
/Users/preetkaria/PycharmProjects/when_to_leave/models/PPO_EnvImpr3_b56d0_2M.zip


## Common Methods

In [4]:
def getResultColumnName(column: str, columns_to_visualize: list, type: str) -> str:
    for col in columns_to_visualize:
        if type in col and column in col:
            return col
    print(f"No matching column found for {column} with type {type}")

def testFlow(test_df_path: str, models: list):
    # Load test data
    test_df = pd.DataFrame()
    with open(test_df_path, 'rb') as f:
        test_df =pickle.load(f)
        
    # Evaluate models
    
    columns_to_visualize = []
    for model in models:
        print(f'Evaluating {model.prediction_keys()}')
        columns_to_visualize += model.prediction_keys()
        
        new_cols = test_df.parallel_apply(lambda row: model.predict(row, override=False), result_type='expand', axis=1)
        for i, col_name in enumerate(model.prediction_keys()):
            test_df[col_name] = new_cols[i]
            print(col_name, test_df[col_name].mean())
            
    # Save the test df with predictions
    test_df_path = Path(test_df_path)
    test_df.to_csv(test_df_path.with_suffix('.csv'), index=False)
    test_df.to_pickle(test_df_path)
    
    return test_df, columns_to_visualize

def evaluateResults(test_df: pd.DataFrame, columns_to_visualize: list, test_name: str, 
                    cost_results_df: pd.DataFrame, observed_n_results_df: pd.DataFrame):
    means = test_df[columns_to_visualize].mean()
    cost_results_row = {'Test Name': test_name}
    for cost_column in cost_results_df.columns:
        if cost_column == 'Test Name':
            continue
        mean_col = getResultColumnName(cost_column, columns_to_visualize, 'cost')
        cost_results_row[cost_column] = means.get(mean_col)
    cost_results_df = pd.concat([cost_results_df, pd.DataFrame([cost_results_row])], ignore_index=True)
    
    observed_n_results_rows = {'Test Name': test_name}
    for observed_n_column in observed_n_results_df.columns:
        if observed_n_column == 'Test Name':
            continue
        mean_col = getResultColumnName(observed_n_column, columns_to_visualize, 'observed_n')
        observed_n_results_rows[observed_n_column] = means.get(mean_col)
    observed_n_results_df = pd.concat([observed_n_results_df, pd.DataFrame([observed_n_results_rows])], ignore_index=True)
    
    return cost_results_df, observed_n_results_df

## Test on Simulated Data

In [5]:
base_sim_test = OrderedDict()
base_sim_test['low alpha from 0.1 to 1.0'] = {
    'alpha_range': np.arange(0.1, 1.0, 0.1).tolist(),
    'beta_range': np.arange(1.1, 2.0, 0.1).tolist(),
    'h_range': [0.5],
    'c_range': [25],
    'total': list(range(10, 40)),
    'travel_time': TravelTimeDist.UNIFORM,
    'param_estimator': PARAM_ESTIMATOR,
}

base_sim_test['medium alpha from 1.0 to 3.0'] = {
    'alpha_range': np.arange(1.0, 3.0, 0.1).tolist(),
    'beta_range': np.arange(1.1, 2.0, 0.1).tolist(),
    'h_range': [0.5],
    'c_range': [25],
    'total': list(range(10, 40)),
    'travel_time': TravelTimeDist.UNIFORM,
    'param_estimator': PARAM_ESTIMATOR,
}

base_sim_test['high alpha from 3.0 to 5.0'] = {
    'alpha_range': np.arange(3.0, 5.0, 0.1).tolist(),
    'beta_range': np.arange(1.1, 2.0, 0.1).tolist(),
    'h_range': [0.5],
    'c_range': [25],
    'total': list(range(10, 40)),
    'travel_time': TravelTimeDist.UNIFORM,
    'param_estimator': PARAM_ESTIMATOR,
}

base_sim_test['high beta from 3 to 5'] = {
    'alpha_range': np.arange(0.1, 3.0, 0.1).tolist(),
    'beta_range': np.arange(3.0, 5.0, 0.1).tolist(),
    'h_range': [0.5],
    'c_range': [25],
    'total': list(range(10, 40)),
    'travel_time': TravelTimeDist.UNIFORM,
    'param_estimator': PARAM_ESTIMATOR,
}
sim_tests = OrderedDict()
for travel_time in TravelTimeDist:
    for test_name, config in base_sim_test.items():
        new_test_name = f"{test_name} with {travel_time.name.lower()} travel time"
        new_config = config.copy()
        new_config['travel_time'] = travel_time
        sim_tests[new_test_name] = new_config


In [6]:
for test_name, config in sim_tests.items():
    print(f'Running test: {test_name}')
    
    test_df_path = generate_data.generate(config=config, row_count=10_000)
    
    pandarallel.initialize(progress_bar=True)
        
    test_df = pd.DataFrame()
    with open(test_df_path, 'rb') as f:
        test_df =pickle.load(f)
        
    # Evaluate models
    
    columns_to_visualize = []
    for model in user_models + oracle_models:
        print(f'Evaluating {model.prediction_keys()}')
        columns_to_visualize += model.prediction_keys()
        
        new_cols = test_df.parallel_apply(lambda row: model.predict(row, override=False), result_type='expand', axis=1)
        for i, col_name in enumerate(model.prediction_keys()):
            test_df[col_name] = new_cols[i]
            print(col_name, test_df[col_name].mean())
            
    # Save the test df with predictions
    test_df_path = Path(test_df_path)
    test_df.to_csv(test_df_path.with_suffix('.csv'), index=False)
    test_df.to_pickle(test_df_path)

    cost_results_df, observed_n_results_df = evaluateResults(test_df, columns_to_visualize, test_name, cost_results_df, observed_n_results_df)
    
    
        
        
    
    

Running test: low alpha from 0.1 to 1.0 with uniform travel time
[INFO] Dataset already exists:
/Users/preetkaria/PycharmProjects/when_to_leave/data/e3a7b_10K.pkl

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.
Evaluating ['cost_PPO_EnvImpr3_b56d0_2M', 'observed_n_PPO_EnvImpr3_b56d0_2M']


cost_PPO_EnvImpr3_b56d0_2M 4.003231441825221
observed_n_PPO_EnvImpr3_b56d0_2M 6.2107
Evaluating ['cost_PTO_Var_n', 'observed_n_PTO_Var_n']


cost_PTO_Var_n 4.071889132405377
observed_n_PTO_Var_n 5.964
Evaluating ['cost_PTO_Fix_n_3']


cost_PTO_Fix_n_3 7.328311526482266
Evaluating ['cost_PTO_Fix_n_5']


cost_PTO_Fix_n_5 8.14565344747998
Evaluating ['cost_Oracle_Var_n', 'observed_n_Oracle_Var_n']


...

cost_PTO_Fix_n_3 11.625613788506989
Evaluating ['cost_PTO_Fix_n_5']


cost_PTO_Fix_n_5 16.58279290195975
Evaluating ['cost_Oracle_Var_n', 'observed_n_Oracle_Var_n']


cost_Oracle_Var_n 10.97996024482841
observed_n_Oracle_Var_n 2.3684
Evaluating ['cost_Oracle_Fix_n_3']


cost_Oracle_Fix_n_3 8.532635319571167
Evaluating ['cost_Oracle_Fix_n_5']


cost_Oracle_Fix_n_5 16.236826292270205


## Test Prof Data

In [7]:
test_df_path = Path(methods.file_path('prof_full_data.pkl', dir_name='data/prof_data_test/'))
test_name = "Prof Data Test"

pandarallel.initialize(progress_bar=True)
        
test_df = pd.DataFrame()
with open(test_df_path, 'rb') as f:
    test_df =pickle.load(f)


# Evaluate models

columns_to_visualize = []
for model in user_models:
    print(f'Evaluating {model.prediction_keys()}')
    columns_to_visualize += model.prediction_keys()
    
    new_cols = test_df.parallel_apply(lambda row: model.predict(row, override=False), result_type='expand', axis=1)
    for i, col_name in enumerate(model.prediction_keys()):
        test_df[col_name] = new_cols[i]
        print(col_name, test_df[col_name].mean())
        
# Save the test df with predictions
test_df_path = Path(test_df_path)
test_df.to_csv(test_df_path.with_suffix('.csv'), index=False)
test_df.to_pickle(test_df_path)

cost_results_df, observed_n_results_df = evaluateResults(test_df, columns_to_visualize, test_name, cost_results_df, observed_n_results_df)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.
Evaluating ['cost_PPO_EnvImpr3_b56d0_2M', 'observed_n_PPO_EnvImpr3_b56d0_2M']


cost_PPO_EnvImpr3_b56d0_2M 14.077771008227568
observed_n_PPO_EnvImpr3_b56d0_2M 16.526
Evaluating ['cost_PTO_Var_n', 'observed_n_PTO_Var_n']


cost_PTO_Var_n 14.274019641719478
observed_n_PTO_Var_n 16.1669
Evaluating ['cost_PTO_Fix_n_3']


cost_PTO_Fix_n_3 26.405796761524012
Evaluating ['cost_PTO_Fix_n_5']


cost_PTO_Fix_n_5 22.468632054456194
No matching column found for Oracle_Var_n with type cost
No matching column found for Oracle_Fix_n_3 with type cost
No matching column found for Oracle_Fix_n_5 with type cost
No matching column found for Oracle_Var_n with type observed_n


/var/folders/6b/tmvvscrj5wb5r4ngnndf1gtm0000gn/T/ipykernel_15846/1811744699.py:41: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  cost_results_df = pd.concat([cost_results_df, pd.DataFrame([cost_results_row])], ignore_index=True)
/var/folders/6b/tmvvscrj5wb5r4ngnndf1gtm0000gn/T/ipykernel_15846/1811744699.py:49: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  observed_n_results_df = pd.concat([observed_n_results_df, pd.DataFrame([observed_n_results_rows])], ignore_index=True)


## Test Bicycle Data

In [8]:
test_df_path = Path(methods.file_path('bicycle_data.pkl', dir_name='data/non_gamma_bicyle_test'))
test_name = "Bicycle Data Test"

pandarallel.initialize(progress_bar=True)
        
test_df = pd.DataFrame()
with open(test_df_path, 'rb') as f:
    test_df =pickle.load(f)
    
# Evaluate models

columns_to_visualize = []
for model in user_models:
    print(f'Evaluating {model.prediction_keys()}')
    columns_to_visualize += model.prediction_keys()
    
    new_cols = test_df.parallel_apply(lambda row: model.predict(row, override=False), result_type='expand', axis=1)
    for i, col_name in enumerate(model.prediction_keys()):
        test_df[col_name] = new_cols[i]
        print(col_name, test_df[col_name].mean())
        
# Save the test df with predictions
test_df_path = Path(test_df_path)
test_df.to_csv(test_df_path.with_suffix('.csv'), index=False)
test_df.to_pickle(test_df_path)

cost_results_df, observed_n_results_df = evaluateResults(test_df, columns_to_visualize, test_name, cost_results_df, observed_n_results_df)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.
Evaluating ['cost_PPO_EnvImpr3_b56d0_2M', 'observed_n_PPO_EnvImpr3_b56d0_2M']


cost_PPO_EnvImpr3_b56d0_2M 18.362613762074606
observed_n_PPO_EnvImpr3_b56d0_2M 36.077777777777776
Evaluating ['cost_PTO_Var_n', 'observed_n_PTO_Var_n']


cost_PTO_Var_n 19.143514138555556
observed_n_PTO_Var_n 35.44444444444444
Evaluating ['cost_PTO_Fix_n_3']


cost_PTO_Fix_n_3 34.33659797694444
Evaluating ['cost_PTO_Fix_n_5']


cost_PTO_Fix_n_5 29.870037397500003
No matching column found for Oracle_Var_n with type cost
No matching column found for Oracle_Fix_n_3 with type cost
No matching column found for Oracle_Fix_n_5 with type cost
No matching column found for Oracle_Var_n with type observed_n


/var/folders/6b/tmvvscrj5wb5r4ngnndf1gtm0000gn/T/ipykernel_15846/1811744699.py:41: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  cost_results_df = pd.concat([cost_results_df, pd.DataFrame([cost_results_row])], ignore_index=True)
/var/folders/6b/tmvvscrj5wb5r4ngnndf1gtm0000gn/T/ipykernel_15846/1811744699.py:49: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  observed_n_results_df = pd.concat([observed_n_results_df, pd.DataFrame([observed_n_results_rows])], ignore_index=True)


In [9]:
display(cost_results_df.head())
display(observed_n_results_df.head())
cost_results_df.to_csv(methods.file_path('finla_test_cost_results.csv', dir_name='results'), index=False)
observed_n_results_df.to_csv(methods.file_path('finla_test_observed_n_results.csv', dir_name='results'), index=False)

,Test Name,PPO,PTO_Var_n,Oracle_Var_n,PTO_Fix_n_3,Oracle_Fix_n_3,PTO_Fix_n_5,Oracle_Fix_n_5
0,low alpha from 0.1 to 1.0 with uniform travel ...,4.003231,4.071889,3.371634,7.328312,3.364770,8.145653,5.801535
1,medium alpha from 1.0 to 3.0 with uniform trav...,8.101072,8.124943,6.787735,13.557500,7.423973,13.084960,9.112320
2,high alpha from 3.0 to 5.0 with uniform travel...,11.025299,10.909151,8.791283,17.319892,10.082228,16.366855,11.629784
3,high beta from 3 to 5 with uniform travel time,13.778456,13.898372,11.612308,19.311932,12.750842,18.057125,13.735757
4,low alpha from 0.1 to 1.0 with low travel time,4.991679,5.200731,4.206075,9.483396,5.153306,8.541790,5.481455


,Test Name,PPO,PTO_Var_n,Oracle_Var_n
0,low alpha from 0.1 to 1.0 with uniform travel ...,6.2107,5.9640,4.4958
1,medium alpha from 1.0 to 3.0 with uniform trav...,10.1598,9.8417,9.4736
2,high alpha from 3.0 to 5.0 with uniform travel...,11.3860,10.9297,10.7934
3,high beta from 3 to 5 with uniform travel time,9.8349,9.3166,9.3768
4,low alpha from 0.1 to 1.0 with low travel time,10.1325,9.6539,9.3594
